This notebooks demonstrates how to convince spikeinterface to load externally preprocessed data

In [ ]:
!pip install spikeinterface

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root = "/content/drive/MyDrive/datasai-daw/data/2021-07-20_11-59-01"
src = Path(root) / "Record Node 115"

In [ ]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.widgets as sw
from probeinterface import Probe
from probeinterface.plotting import plot_probe

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [ ]:
rec = se.read_openephys(src, stream_name="Record Node 115#Neuropix-PXI-111.0")

Now we have the raw data loaded. Let's see what we have.

In [ ]:
channel_ids = rec.get_channel_ids()
fs_Hz = rec.get_sampling_frequency()
num_chans = rec.get_num_channels()
num_segments = rec.get_num_segments()

print(f'Channel ids: {channel_ids[:5]} {channel_ids[-5:]}')
print(f'Sampling frequency: {fs_Hz}')
print(f'Number of channels: {num_chans}')
print(f"Number of segments: {num_segments}")

If you have more than one "experiment" or "recording" inside your openephys data folder, `rec` will comprise multiple "segments", one per recording. You can select a single recording like this:

    rec1 =

Here, we have only one segment, so we proceed.

First, we extract a short snippet, so we can visualize some raw data:

In [ ]:
dat_snippet = rec.get_traces(start_frame=int(0.0*fs_Hz), end_frame=int(2.0*fs_Hz))
print('Snippet shape:', dat_snippet.shape)
plt.plot(dat_snippet[:,1])

Next: Did spikeinterface understand our probe geometry?

In [ ]:
probe = rec.get_probe()
print(probe)

That's a hard no.

We'll have to build our own probe representation.

In [ ]:
C = dat_snippet.shape[1]
xy = []
xx0 = [43, 11, 59, 27] # From "NeuropixPhase3A_kilosortChanMap.mat"
for c in range(C):
    x = xx0[c%len(xx0)]
    y = 20 + 20*(c//2) # Ibid
    xy.append([x,y])
positions = np.array(xy)
positions[:10]

Geometry looks good. Let's assemble.

In [ ]:
probe = Probe(ndim=2, si_units='um')
probe.set_contacts(positions=positions, shapes='square', shape_params={'width': 12, 'height': 12})
# Shape info from https://www.neuropixels.org/_files/ugd/832f20_4a14406ba1204e60ae8534b09e201b49.pdf
probe.create_auto_shape()
probe.set_device_channel_indices(np.arange(C))
plot_probe(probe, with_channel_index=True)

Hard to see the details. Good to practice with `C=12` first. Try it.

Attaching the probe to the recording oddly involves copying the recording object:

In [ ]:
rec1 = rec.set_probe(probe)

(Though no actual data gets copied.)

Next, we set up a simple filtering pipeline:

In [ ]:
rec_car = spre.common_reference(rec1, reference='local', operator='average', local_radius=(0,100))

and, since we are only creating a placeholder to receive our SALPA data, pick a small slice of data:*italicized text*

In [ ]:
rec_sub = rec_car.frame_slice(start_frame=0.0*fs_Hz, end_frame=2.0*60*fs_Hz) # grab 2 minutes

Now, we can construct a preprocessed output

In [ ]:
job_kwargs = dict(n_jobs=10, chunk_duration="1s", progress_bar=True)
rec_sub.save(folder=src / "preproc", **job_kwargs)

This will construct a folder called "preproc" inside the "Record Node 115" subfolder of our experiment. Inside it are many files that explain our pipeline so far, but they are not critical. The only file that matters is "traces_cached_seg0.raw". This we can simply replace with the "salpa.dat" file from our own pipeline.

(In fact, for the tutorial, I copied the "preproc" folder as "salpa", and replaced the "traces_cached_seg0.raw" file in there, keeping the original "preproc" folder for future reference.)